In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd
import os
os.chdir('/content/drive/MyDrive/Project4/')
!pwd

/content
/content/drive/MyDrive/Project4


In [ ]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
import spacy
import gensim

# Read Dataset 1

In [ ]:
df1 = pd.read_csv('process_data/df1_Add_era_country.csv')

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54944 entries, 0 to 54943
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   poem_id     54944 non-null  int64 
 1   poet_name   54944 non-null  object
 2   poem_title  54944 non-null  object
 3   poem_text   54944 non-null  object
 4   era         54944 non-null  object
 5   country     13409 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.5+ MB


In [ ]:
df1.head()

,poem_id,poet_name,poem_title,poem_text,era,country
0,0,خلفان بن مصبح,بدت تختال في حُلل الجمالِ,بدت تختال في حُلل الجمالِ\nوجادت بالزيارة والو...,العصر الحديث,الإمارات
1,1,خلفان بن مصبح,يا طائر الشعر القرير,يا طائر الشعر القرير\nيا وحي إلهام الصدور\nأسع...,العصر الحديث,الإمارات
2,2,خلفان بن مصبح,بنت حجرات أرى من عجب,بنت حجرات أرى من عجب\nأن أرى فيك جمال العرب\nد...,العصر الحديث,الإمارات
3,3,خلفان بن مصبح,هذا الربيع بنور الحسن وافانا,هذا الربيع بنور الحسن وافانا\nوقد كسا الأرض با...,العصر الحديث,الإمارات
4,4,خلفان بن مصبح,روحي فداك وإن مُنحتُ صدوداً,روحي فداك وإن مُنحتُ صدوداً\nأخفاك ما بي أم أط...,العصر الحديث,الإمارات


In [ ]:
df1.shape

(54944, 6)

In [ ]:
df1.columns

Index(['poem_id', 'poet_name', 'poem_title', 'poem_text', 'era', 'country'], dtype='object')

In [ ]:
# for dataset 1
df_new = df1.groupby(['poet_name', 'poem_title'])
df_new.first()

poem_id  \
poet_name      poem_title                               
آمنة بنت عتيبة على مثل ابن مية فانعياه          14914   
آمنة بنت وهب   بارك فيك الله من غلام            14916   
               عفا جانب البطحاء من بن هاشم      14915   
أبان الالحقي   أبا الأطول طولت                  19500   
               ألا قل لعبيد الله                19501   
...                                               ...   
يوسف النبهاني  يا ليتني للحجاز بالغ              7292   
               يا ليته بالمدينة اعتكفا           7293   
               يا نبيا لدى الإله عظيما           7345   
               يا نسيماً سرى إلى قاسيونا         7354   
يوسف بن رويسم  لجناب ذي الإكرام والشرف العلي      151   

                                                                                      poem_text  \
poet_name      poem_title                                                                         
آمنة بنت عتيبة على مثل ابن مية فانعياه        عَلى مثلِ اِبن ميّة فَاِنعياه\nتشقّ نَواعمُ ال...   
آمنة بنت وهب   بارك فيك الله من غلام          بارَكَ فيك اللّهُ من غلامٍ\nيا اِبن الّذي في ح...   
               عفا جانب البطحاء من بن هاشم    عفا جانب البطحاء من بن هاشم\nوجاور لحدا خارجا ...   
أبان الالحقي   أبا الأطول طولت                أَبا الأَطوَلِ طَوَّلتَ\nوَما يَنفَعُ تَطويلُ\...   
               ألا قل لعبيد الله              أَلا قُل لِعُبَيدِ اللَـ\nـهِ ما بالُكَ لا تَس...   
...                                                                                         ...   
يوسف النبهاني  يا ليتني للحجاز بالغ           يا لَيتَني لِلحجازِ بالِغ\nوفيهِ عَيشي يا سعدُ...   
               يا ليته بالمدينة اعتكفا        يا ليتهُ بِالمدينة اِعتكفا\nينالُ فيها الألطاف...   
               يا نبيا لدى الإله عظيما        يا نَبيّاً لَدى الإلهِ عَظيما\nوَحَبيباً لهُ و...   
               يا نسيماً سرى إلى قاسيونا      يا نَسيماً سرى إلى قاسيونا\nحيِّ حَبراً بسفحهِ...   
يوسف بن رويسم  لجناب ذي الإكرام والشرف العلي  لجناب ذي الإكرام والشرف العلي\nوالمجد والمعروف...   

                                                        era   country  
poet_name      poem_title                                              
آمنة بنت عتيبة على مثل ابن مية فانعياه        العصر الجاهلي      None  
آمنة بنت وهب   بارك فيك الله من غلام          العصر الجاهلي      None  
               عفا جانب البطحاء من بن هاشم    العصر الجاهلي      None  
أبان الالحقي   أبا الأطول طولت                العصر العباسي      None  
               ألا قل لعبيد الله              العصر العباسي      None  
...                                                     ...       ...  
يوسف النبهاني  يا ليتني للحجاز بالغ            العصر الحديث    فلسطين  
               يا ليته بالمدينة اعتكفا         العصر الحديث    فلسطين  
               يا نبيا لدى الإله عظيما         العصر الحديث    فلسطين  
               يا نسيماً سرى إلى قاسيونا       العصر الحديث    فلسطين  
يوسف بن رويسم  لجناب ذي الإكرام والشرف العلي   العصر الحديث  الإمارات  

[54571 rows x 4 columns]

# Read Dataset 2

In [ ]:
df2 = pd.read_csv('process_data/df2_Add_era_country.csv')

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58020 entries, 0 to 58019
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   poem_id     58020 non-null  int64 
 1   poet_name   58020 non-null  object
 2   poem_title  58020 non-null  object
 3   poem_text   58020 non-null  object
 4   era         58020 non-null  object
 5   country     27146 non-null  object
 6   poem_style  58020 non-null  object
dtypes: int64(1), object(6)
memory usage: 3.1+ MB


In [ ]:
df2.head()

,poem_id,poet_name,poem_title,poem_text,era,country,poem_style
0,21,بدر شاكر السياب,أنشودة المطر,عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي...,العصر الحديث,العراق,فصحى
1,65546,بدر شاكر السياب,أقداح و أحلام,انا لا ازال و في يدي قدحي ياليل اين تفرق الشر...,العصر الحديث,العراق,فصحى
2,65561,بدر شاكر السياب,هوى واحد !,علي مقلتيك ارتشفت النجوم وعانقت امالي الايبة ...,العصر الحديث,العراق,فصحى
3,65554,بدر شاكر السياب,أساطير,اساطير من حشرجات الزمان نسيج اليد البالية رواه...,العصر الحديث,العراق,فصحى
4,65550,بدر شاكر السياب,اللقاء الأخير,والتف حولك ساعداي ومال جيدك في اشتهاء كالزهرة ...,العصر الحديث,العراق,فصحى


In [ ]:
#show columns for dataset2
df2.columns

Index(['poem_id', 'poet_name', 'poem_title', 'poem_text', 'era', 'country',
       'poem_style'],
      dtype='object')

In [ ]:
# for dataset 2
df_new2 = df2.groupby(['poet_name', 'poem_title'])
df_new2.first()

poem_id  \
poet_name        poem_title                                            
  عبد الله الطيب  إلى الخرطوم                                  65222   
                 أضْعَـفُ الإيمانِ قافيـةٌ                     75645   
                 بين الرياء والحياء                            65221   
                 عُوجوا عَلى تُونسَ الخَضراءِ وَاِبتَهِجوا     71720   
                 لا تسألِ الكذابَ عن نياتِهِ                   73184   
...                                                              ...   
يوسف الخال       يا قمر                                        73059   
يوسف الخطيب      العندليب المهاجر                              81679   
                 القبَرة                                       81680   
                 يوسف                                          81678   
يوسف الصائغ      إجازة                                         63189   

                                                                                                     poem_text  \
poet_name        poem_title                                                                                      
  عبد الله الطيب  إلى الخرطوم                                 الي الخرطوم من بعد اغتراب وبعد بلي الشهي من ا...   
                 أضْعَـفُ الإيمانِ قافيـةٌ                   قصيدة في مدح النبي صلي اله عليه وسلم *** بانت ...   
                 بين الرياء والحياء                           كلما لاح برقها خفق القل ب وجاشت من الحنين الع...   
                 عُوجوا عَلى تُونسَ الخَضراءِ وَاِبتَهِجوا    عوجوا علي تونس الخضراء وابتهجوا واشفوا الظماء...   
                 لا تسألِ الكذابَ عن نياتِهِ                  لا تسال الكذاب عن نياته مادام كذابا عليك لسان...   
...                                                                                                        ...   
يوسف الخال       يا قمر                                      لا تسل عني فماضي طل وغدي انقاضه لا لا تسل انت ...   
يوسف الخطيب      العندليب المهاجر                            اترك مثلي يا رفيق تمر بالزمن عبر اليالي السود ...   
                 القبَرة                                     تلك يا صاح قبرة في الحدود خرقت الف حرمة لعهود ...   
                 يوسف                                        ‏ بريءة منكم فلسطين التي تذبحون‏ و لحمها باق ع...   
يوسف الصائغ      إجازة                                       شهداء عشرة نزلوا يوم اجازتهم لبصرة اربعة منهم ...   

                                                                      era  \
poet_name        poem_title                                                 
  عبد الله الطيب  إلى الخرطوم                                العصر الحديث   
                 أضْعَـفُ الإيمانِ قافيـةٌ                   العصر الحديث   
                 بين الرياء والحياء                          العصر الحديث   
                 عُوجوا عَلى تُونسَ الخَضراءِ وَاِبتَهِجوا   العصر الحديث   
                 لا تسألِ الكذابَ عن نياتِهِ                 العصر الحديث   
...                                                                   ...   
يوسف الخال       يا قمر                                      العصر الحديث   
يوسف الخطيب      العندليب المهاجر                            العصر الحديث   
                 القبَرة                                     العصر الحديث   
                 يوسف                                        العصر الحديث   
يوسف الصائغ      إجازة                                       العصر الحديث   

                                                             country  \
poet_name        poem_title                                            
  عبد الله الطيب  إلى الخرطوم                                السودان   
                 أضْعَـفُ الإيمانِ قافيـةٌ                   السودان   
                 بين الرياء والحياء                          السودان   
                 عُوجوا عَلى تُونسَ الخَضراءِ وَاِبتَهِجوا   السودان   
                 لا تسألِ الكذابَ عن نياتِهِ                 السودان   
...                                                              ...   
يوسف الخال       يا قمر                                        

### Search for poet_name in dataset1 isin poet_name dataset 2

In [ ]:
poet_name_in_df1_condition = df1['poet_name'].isin(df2['poet_name'])
poet_name_in_df1_condition

0        False
1        False
2        False
3        False
4        False
         ...  
54939     True
54940     True
54941     True
54942     True
54943     True
Name: poet_name, Length: 54944, dtype: bool

In [ ]:
poet_name_in_df1 = df1.loc[poet_name_in_df1_condition]
poet_name_in_df1

,poem_id,poet_name,poem_title,poem_text,era,country
152,152,هدى السعدي,يقطرُنا الحنين,يقطرُنا الحنينُ رؤىً\nعلى أهدابِ داليةِ العذاب...,العصر الحديث,الإمارات
153,153,هدى السعدي,هذا دمي,عبلة تواسي المعتقلات العراقيات في سجون الأعداء...,العصر الحديث,الإمارات
154,154,هدى السعدي,سامحني أحبك,لا أعرف أين يكون محلك في دنيايّ\nأو أين يكون م...,العصر الحديث,الإمارات
155,155,هدى السعدي,أول العنت,غضبٌ شديدْ\nيجتاح شريان الوليدْ\nفي عتمة الرحم...,العصر الحديث,الإمارات
156,156,هدى السعدي,أما من غيمة حبلى,أما من غيمة حبلى\nفهذا الجمر تنفخ في مواقده\nر...,العصر الحديث,الإمارات
...,...,...,...,...,...,...
54939,55113,ابن زمرك,اسقني بنت الدوالي,اسقني بنت الدوالي\nإنها نعم الدوالي\nسقّني سر ...,العصر المملوكي,NaN
54940,55114,ابن زمرك,الإلاه يحمد ويشكر,الإلاه يحمد ويشكر\nالمطر من اهن لشكر\nوالإلاه ...,العصر المملوكي,NaN
54941,55115,ابن زمرك,ألني أبلغ من العمل,ألنيّ أبلغ من العمل\nاللّه يبلّغك الأمل\nنزيه ...,العصر المملوكي,NaN
54942,55116,ابن زمرك,مهرجان هوذا اليوم للإنسان,مهرجان هوذا اليوم للإنسان\nألنزّيه والحسن والإ...,العصر المملوكي,NaN


In [ ]:
poet_name_in_df1.head()

,poem_id,poet_name,poem_title,poem_text,era,country
152,152,هدى السعدي,يقطرُنا الحنين,يقطرُنا الحنينُ رؤىً\nعلى أهدابِ داليةِ العذاب...,العصر الحديث,الإمارات
153,153,هدى السعدي,هذا دمي,عبلة تواسي المعتقلات العراقيات في سجون الأعداء...,العصر الحديث,الإمارات
154,154,هدى السعدي,سامحني أحبك,لا أعرف أين يكون محلك في دنيايّ\nأو أين يكون م...,العصر الحديث,الإمارات
155,155,هدى السعدي,أول العنت,غضبٌ شديدْ\nيجتاح شريان الوليدْ\nفي عتمة الرحم...,العصر الحديث,الإمارات
156,156,هدى السعدي,أما من غيمة حبلى,أما من غيمة حبلى\nفهذا الجمر تنفخ في مواقده\nر...,العصر الحديث,الإمارات


In [ ]:
list_poet_name = poet_name_in_df1.poet_name.unique()

In [ ]:
print(sorted(list_poet_name))

['أبو الأسود الدؤلي', 'أبو البقاء الرندي', 'أبو الشمقمق', 'أبو القاسم الشابي', 'أبو الهدى الصيادي', 'أبو بكر الخوارزمي', 'أبو تمام', 'أبو طالب', 'أبو فراس الحمداني', 'أحلام مستغانمي', 'أحمد شوقي', 'أحمد محرم', 'أسامة بن منقذ', 'أمل دنقل', 'أوس بن حجر', 'إبراهيم الأسطى', 'إبراهيم طوقان', 'إبراهيم عبد القادر المازني', 'ابن الخيمي', 'ابن الرومي', 'ابن الفارض', 'ابن المعتز', 'ابن حجاج', 'ابن حيوس', 'ابن زريق البغدادي', 'ابن زمرك', 'ابن زيدون', 'ابن شيخان السالمي', 'ابن عبد ربه', 'ابن عنين', 'ابن معتوق', 'الأبيوردي', 'الأخطل', 'الأعشى', 'الإمام الشافعي', 'الباخرزي', 'البحتري', 'البرعي', 'البوصيري', 'الحارث بن حلزة', 'الحطيئة', 'الخرنق بنت بدر', 'الخنساء', 'الراعي النميري', 'السري الرفاء', 'السموأل', 'الشاب الظريف', 'الشريف الرضي', 'الشريف المرتضى', 'الشنفرى', 'الطرماح', 'الفرزدق', 'المتلمس الضبعي', 'المتنبي', 'المثقب العبدي', 'المعتمد بن عباد', 'النابغة الذبياني', 'الهبل', 'امرؤ القيس', 'بدر شاكر السياب', 'بديع الزمان الهمذاني', 'بشار بن برد', 'بهاء الدين زهير', 'تميم البرغوثي', 'توفيق زياد

In [ ]:
len(list_poet_name)

114

In [ ]:
# save poet_name repeated in dataset 1 to file csv
poet_name_in_df1.to_csv("process_data/poet_name_in_df1.csv" ,  encoding='utf-8-sig', index=False , header=True)

### Search for poet_name in dataset2 isin poet_name dataset 1

In [ ]:
poet_name_in_df2_condition = df2['poet_name'].isin(df1['poet_name'])
poet_name_in_df2_condition

0         True
1         True
2         True
3         True
4         True
         ...  
58015    False
58016    False
58017    False
58018    False
58019    False
Name: poet_name, Length: 58020, dtype: bool

In [ ]:
poet_name_in_df2 = df2.loc[poet_name_in_df2_condition]
poet_name_in_df2

,poem_id,poet_name,poem_title,poem_text,era,country,poem_style
0,21,بدر شاكر السياب,أنشودة المطر,عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي...,العصر الحديث,العراق,فصحى
1,65546,بدر شاكر السياب,أقداح و أحلام,انا لا ازال و في يدي قدحي ياليل اين تفرق الشر...,العصر الحديث,العراق,فصحى
2,65561,بدر شاكر السياب,هوى واحد !,علي مقلتيك ارتشفت النجوم وعانقت امالي الايبة ...,العصر الحديث,العراق,فصحى
3,65554,بدر شاكر السياب,أساطير,اساطير من حشرجات الزمان نسيج اليد البالية رواه...,العصر الحديث,العراق,فصحى
4,65550,بدر شاكر السياب,اللقاء الأخير,والتف حولك ساعداي ومال جيدك في اشتهاء كالزهرة ...,العصر الحديث,العراق,فصحى
...,...,...,...,...,...,...,...
57980,86646,المعتمد بن عباد,صَنَعَ الرّيحُ من الماءِ زَرَد,صنع الريح من الماء زرد (اي درع لقتال لو جمد),العصر الأندلسي,NaN,فصحى
57981,86647,المعتمد بن عباد,كأنَّها فوقَ العصا,كانها فوق العصا (هامة زنجي عصي),العصر الأندلسي,NaN,فصحى
57982,86648,المعتمد بن عباد,انظرهما في الظّلام قد نجَما,انظرهما في الظلام قد نجما* انظرهما في الظلام ق...,العصر الأندلسي,NaN,فصحى
57983,86649,المعتمد بن عباد,هذا المؤذِّن قد بدا بأذانه,هذا المءذن قد بدا باذانه* هذا المءذن قد بدا با...,العصر الأندلسي,NaN,فصحى


In [ ]:
poet_name_in_df2.head()

,poem_id,poet_name,poem_title,poem_text,era,country,poem_style
0,21,بدر شاكر السياب,أنشودة المطر,عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي...,العصر الحديث,العراق,فصحى
1,65546,بدر شاكر السياب,أقداح و أحلام,انا لا ازال و في يدي قدحي ياليل اين تفرق الشر...,العصر الحديث,العراق,فصحى
2,65561,بدر شاكر السياب,هوى واحد !,علي مقلتيك ارتشفت النجوم وعانقت امالي الايبة ...,العصر الحديث,العراق,فصحى
3,65554,بدر شاكر السياب,أساطير,اساطير من حشرجات الزمان نسيج اليد البالية رواه...,العصر الحديث,العراق,فصحى
4,65550,بدر شاكر السياب,اللقاء الأخير,والتف حولك ساعداي ومال جيدك في اشتهاء كالزهرة ...,العصر الحديث,العراق,فصحى


In [ ]:
list_poet_name2 = poet_name_in_df2.poet_name.unique()

In [ ]:
print (sorted(list_poet_name2))

['أبو الأسود الدؤلي', 'أبو البقاء الرندي', 'أبو الشمقمق', 'أبو القاسم الشابي', 'أبو الهدى الصيادي', 'أبو بكر الخوارزمي', 'أبو تمام', 'أبو طالب', 'أبو فراس الحمداني', 'أحلام مستغانمي', 'أحمد شوقي', 'أحمد محرم', 'أسامة بن منقذ', 'أمل دنقل', 'أوس بن حجر', 'إبراهيم الأسطى', 'إبراهيم طوقان', 'إبراهيم عبد القادر المازني', 'ابن الخيمي', 'ابن الرومي', 'ابن الفارض', 'ابن المعتز', 'ابن حجاج', 'ابن حيوس', 'ابن زريق البغدادي', 'ابن زمرك', 'ابن زيدون', 'ابن شيخان السالمي', 'ابن عبد ربه', 'ابن عنين', 'ابن معتوق', 'الأبيوردي', 'الأخطل', 'الأعشى', 'الإمام الشافعي', 'الباخرزي', 'البحتري', 'البرعي', 'البوصيري', 'الحارث بن حلزة', 'الحطيئة', 'الخرنق بنت بدر', 'الخنساء', 'الراعي النميري', 'السري الرفاء', 'السموأل', 'الشاب الظريف', 'الشريف الرضي', 'الشريف المرتضى', 'الشنفرى', 'الطرماح', 'الفرزدق', 'المتلمس الضبعي', 'المتنبي', 'المثقب العبدي', 'المعتمد بن عباد', 'النابغة الذبياني', 'الهبل', 'امرؤ القيس', 'بدر شاكر السياب', 'بديع الزمان الهمذاني', 'بشار بن برد', 'بهاء الدين زهير', 'تميم البرغوثي', 'توفيق زياد

In [ ]:
len(list_poet_name2)

114

In [ ]:
# save poet_name repeated in dataset 2 to file csv
poet_name_in_df2.to_csv("process_data/poet_name_in_df2.csv" ,  encoding='utf-8-sig', index=False , header=True)

# Use AraVec Spacy Model  (similarity)

https://github.com/bakrianoo/aravec/blob/master/aravec-with-spacy.ipynb

In [ ]:
# Clean/Normalize Arabic Text
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']

    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)

    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)

    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')

    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])

    #trim
    text = text.strip()

    return text

## Run Your AraVec Spacy Model

In [ ]:
# load AraVec Spacy model
nlp = spacy.load("spacyModel/output/")

In [ ]:
# Define the preprocessing Class
class Preprocessor:
    def __init__(self, tokenizer, **cfg):
        self.tokenizer = tokenizer

    def __call__(self, text):
        preprocessed = clean_str(text)
        return self.tokenizer(preprocessed)

In [ ]:
# Apply the `Preprocessor` Class
nlp.tokenizer = Preprocessor(nlp.tokenizer)

### Search for poem_text if repeated or not is repeat poet_name in df2 , dataset 1

In [ ]:
df3 = pd.read_csv('process_data/poet_name_in_df1.csv')

In [ ]:
df3.shape

(21847, 6)

In [ ]:
df4 = pd.read_csv('process_data/poet_name_in_df2.csv')

In [ ]:
df4.shape

(23876, 7)

In [ ]:
# for first run we test if file is found
# we add the new clean to the file (poet_name_in_df1_Clean)

In [ ]:
# create dataframe for copy data is not repeat
# Create the pandas DataFrame with column name is provided explicitly
path_clean = "process_data/poet_name_in_df1_Clean.csv"

if  os.path.isfile(path_clean):
    poet_name_in_df1_Clean = pd.read_csv(path_clean)
    print (poet_name_in_df1_Clean.shape)
else:

    poet_name_in_df1_Clean = pd.DataFrame( columns=poet_name_in_df1.columns)
    # print dataframe.
    print (poet_name_in_df1_Clean)

Empty DataFrame
Columns: [poem_id, poet_name, poem_title, poem_text, era, country]
Index: []


In [ ]:
poet_name_in_df1_Clean

,poem_id,poet_name,poem_title,poem_text,era,country


In [ ]:
# create dataframe for copy data is not repeat
# Create the pandas DataFrame with column name is provided explicitly
path_repeat = "process_data/poet_name_in_df1_repeat.csv"

if  os.path.isfile(path_repeat):
    poet_name_in_df1_repeat = pd.read_csv(path_repeat)
    print (poet_name_in_df1_Clean.shape)
else:
    poet_name_in_df1_repeat = pd.DataFrame( columns=poet_name_in_df1.columns)
    # print dataframe.
    print(poet_name_in_df1_repeat)

(0, 6)


In [ ]:
poet_name_in_df1_repeat.head()

,poem_id,poet_cat,poet_name,poem_title,poem_text,verses_count
0,152,الإمارات,هدى السعدي,يقطرُنا الحنين,يقطرُنا الحنينُ رؤىً\nعلى أهدابِ داليةِ العذاب...,32.0
1,153,الإمارات,هدى السعدي,هذا دمي,عبلة تواسي المعتقلات العراقيات في سجون الأعداء...,29.5
2,154,الإمارات,هدى السعدي,سامحني أحبك,لا أعرف أين يكون محلك في دنيايّ\nأو أين يكون م...,20.0
3,155,الإمارات,هدى السعدي,أول العنت,غضبٌ شديدْ\nيجتاح شريان الوليدْ\nفي عتمة الرحم...,43.0
4,156,الإمارات,هدى السعدي,أما من غيمة حبلى,أما من غيمة حبلى\nفهذا الجمر تنفخ في مواقده\nر...,27.0


In [ ]:
# search in dataset2 for poem_text if found by similarity is equal=1.0
# between them and poem_text dataset_compare
# you'r save the row is repeated.

In [ ]:
p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')

In [ ]:
count_poet_name_list = []
count_poem_title_list = []
count_poem_text_list = []
start = 0
u = 1
count = 0

#for index3, row in tqdm(df1.iterrows(), total=df1.shape[0]):
for index3 in range(len(df3)):
    if index3 < start:
        continue
    poem_name3     = df3['poet_name'].loc[index3]
    poem_title3    = df3['poem_title'].loc[index3]
    poem_text3     = df3['poem_text'].loc[index3]
    poem_text_nlp3 = nlp(clean_str(poem_text3))
    print ("-"*80 , index3)
    print(poem_name3," ",poem_title3 ," " , poem_text3[:20])

    flag = True
    count_poet_name = 0
    count_poem_title = 0
    count_poem_text =0

    for index4 in range(len(df4)):
        poem_name4  = df4['poet_name'].loc[index4]
        poem_title4 = df4['poem_title'].loc[index4]
        poem_text4  = df4['poem_text'].loc[index4]

        text1 = re.sub(p_tashkeel,"", poem_name3)
        text2 = re.sub(p_tashkeel,"", poem_name4)
        #z=text3.__eq__(text4)
        #z1=re.match(text1, text2,re.IGNORECASE)
        if re.match(text1, text2,re.IGNORECASE):
            count_poet_name += 1
            text3 = re.sub(p_tashkeel,"", poem_title3)
            text4 = re.sub(p_tashkeel,"", poem_title4)
            if re.match(text3, text4,re.IGNORECASE):
                count_poem_title += 1
                poem_text_nlp4 = nlp(clean_str(poem_text4))
                similaritry =  poem_text_nlp3.similarity(poem_text_nlp4)
                if similaritry > 0.97:
                    count_poem_text += 1
                    print(poem_name4," ",poem_title4 ," " , poem_text4[:20])
                    print ("Reapeated found have similarity =" , similaritry ,"in index =" , index4)
                    poet_name_in_df1_repeat.loc[len(poet_name_in_df1_repeat)] = df3.loc[index3]
                    flag = False

                    #break

        if not flag :
            break
    count_poet_name_list.append(count_poet_name)
    count_poem_title_list.append(count_poem_title)
    count_poem_text_list.append(count_poem_text)
    if flag :
        poet_name_in_df1_Clean.loc[len(poet_name_in_df1_Clean)] = df3.loc[index3]
    if index3 == (start + 9 ):
        break

-------------------------------------------------------------------------------- 0
هدى السعدي   يقطرُنا الحنين   يقطرُنا الحنينُ رؤىً
هدى السعدي   يقطِّرنا الحنين    يقطرنا الحنين رءي عل
Reapeated found have similarity = 0.9990085013453328 in index = 23703
-------------------------------------------------------------------------------- 1
هدى السعدي   هذا دمي   عبلة تواسي المعتقلات
هدى السعدي   هذا دمي    عبلة تواسي المعتقلات
Reapeated found have similarity = 0.9996749040703786 in index = 23704
-------------------------------------------------------------------------------- 2
هدى السعدي   سامحني أحبك   لا أعرف أين يكون محل
هدى السعدي   سامحني أحبك    لا اعرف اين يكون محل
Reapeated found have similarity = 0.9987717918169852 in index = 23705
-------------------------------------------------------------------------------- 3
هدى السعدي   أول العنت   غضبٌ شديدْ
يجتاح شري
هدى السعدي   أول العنت    غضب شديد يجتاح شريان
Reapeated found have similarity = 0.9965126114093654 in index = 23706
-----

### save poet_name_in_df1_Clean  to file csv

In [ ]:
poet_name_in_df1_Clean.shape

(0, 6)

In [ ]:
poet_name_in_df1_Clean.head()

,poem_id,poet_name,poem_title,poem_text,era,country


In [ ]:
#save poet_name_in_df1_Clean  to file csv
poet_name_in_df1_Clean.to_csv("process_data/poet_name_in_df1_Clean.csv", encoding='utf-8-sig', index=False, header=True)

### save poet_name_in_df1_repeat to file csv

In [ ]:
poet_name_in_df1_repeat.shape

(11329, 6)

In [ ]:
poet_name_in_df1_repeat.head()

,poem_id,poet_cat,poet_name,poem_title,poem_text,verses_count
0,152,الإمارات,هدى السعدي,يقطرُنا الحنين,يقطرُنا الحنينُ رؤىً\nعلى أهدابِ داليةِ العذاب...,32.0
1,153,الإمارات,هدى السعدي,هذا دمي,عبلة تواسي المعتقلات العراقيات في سجون الأعداء...,29.5
2,154,الإمارات,هدى السعدي,سامحني أحبك,لا أعرف أين يكون محلك في دنيايّ\nأو أين يكون م...,20.0
3,155,الإمارات,هدى السعدي,أول العنت,غضبٌ شديدْ\nيجتاح شريان الوليدْ\nفي عتمة الرحم...,43.0
4,156,الإمارات,هدى السعدي,أما من غيمة حبلى,أما من غيمة حبلى\nفهذا الجمر تنفخ في مواقده\nر...,27.0


In [ ]:
#save poet_name_in_df1_repeat to file csv
poet_name_in_df1_repeat.to_csv("process_data/poet_name_in_df1_repeat.csv", encoding='utf-8-sig', index=False, header=True)